In [12]:
import os
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.optim import Adam
import torch.optim as optim  # for NAdam
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import WandbLogger
from sklearn.model_selection import StratifiedShuffleSplit
from pytorch_lightning.callbacks import EarlyStopping

# -------------------------------
# Helper: Activation Function Getter
# -------------------------------
def get_activation(activation):
    """
    Returns an activation function based on the given input.
    Accepts a string (case insensitive) or a callable.

    Available options:
      "none" (for no activation),
      "relu", "tanh", "sigmoid", "leaky_relu", "gelu", "silu", "mish", "selu".
    For "mish": mish(x) = x * tanh(softplus(x))
    """
    if isinstance(activation, str):
        act = activation.lower()
        if act == "none":
            return lambda x: x  # Identity function (no activation)
        activation_dict = {
            'relu': F.relu,
            'tanh': torch.tanh,
            'sigmoid': torch.sigmoid,
            'leaky_relu': F.leaky_relu,
            'gelu': F.gelu,
            'silu': F.silu,
            'mish': lambda x: x * torch.tanh(F.softplus(x)),
            'selu': F.selu
        }
        return activation_dict.get(act, F.relu)
    return activation

# -------------------------------
# Model Definition: iNaturalistCNN
# -------------------------------
class iNaturalistCNN(pl.LightningModule):
    def __init__(self, num_classes=10, learning_rate=0.001, input_size=128,
                 conv_configs=[(32, 3), (64, 3), (128, 3), (256, 3), (512, 3)],
                 dense_neurons=512,
                 dropout_rate=0.5,
                 conv_activation='relu',
                 dense_activation='relu',
                 use_batch_norm=False,
                 optimizer_type="adam"):
        """
        Args:
            num_classes (int): Number of target classes.
            learning_rate (float): Learning rate.
            input_size (int): Image size (e.g. 224).
            conv_configs (list of tuples): List of 5 tuples, each (num_filters, kernel_size).
            dense_neurons (int): Number of neurons in the dense layer.
            dropout_rate (float): Dropout probability.
            conv_activation (str/callable): Activation for conv layers.
            dense_activation (str/callable): Activation for the dense layer.
            use_batch_norm (bool): Whether to include batch normalization.
            optimizer_type (str): Type of optimizer: "adam" or "nadam".
        """
        super(iNaturalistCNN, self).__init__()
        self.save_hyperparameters(ignore=['conv_activation', 'dense_activation'])
        self.use_batch_norm = use_batch_norm

        # Set activation functions.
        self.conv_activation = get_activation(conv_activation)
        self.dense_activation = get_activation(dense_activation)
        self.dropout_rate = dropout_rate

        # Build the convolutional layers.
        if len(conv_configs) != 5:
            raise ValueError("conv_configs must be a list of exactly five tuples.")
        self.conv_layers = nn.ModuleList()
        if self.use_batch_norm:
            self.bn_layers = nn.ModuleList()
        in_channels = 3  # for RGB images
        for out_channels, k_size in conv_configs:
            conv_layer = nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=k_size,
                stride=1,
                padding=k_size // 2  # maintain spatial dimensions
            )
            self.conv_layers.append(conv_layer)
            if self.use_batch_norm:
                self.bn_layers.append(nn.BatchNorm2d(out_channels))
            in_channels = out_channels

        # Common max-pooling and dropout for convolutional blocks.
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout_conv = nn.Dropout2d(p=dropout_rate)

        # Compute flattened feature dimension using a dummy input.
        dummy_input = torch.zeros(1, 3, input_size, input_size)
        x = dummy_input
        for i, conv in enumerate(self.conv_layers):
            x = conv(x)
            if self.use_batch_norm:
                x = self.bn_layers[i](x)
            x = self.conv_activation(x)
            x = self.pool(x)
            x = self.dropout_conv(x)
        self.feature_dim = x.view(1, -1).size(1)

        # Fully-connected (dense) layers.
        self.fc1 = nn.Linear(self.feature_dim, dense_neurons)
        self.dropout_fc = nn.Dropout(p=dropout_rate)
        self.fc2 = nn.Linear(dense_neurons, 10)

    def forward(self, x):
        # Convolutional blocks: conv -> (optional BN) -> activation -> pool -> dropout.
        for i, conv in enumerate(self.conv_layers):
            x = conv(x)
            if self.use_batch_norm:
                x = self.bn_layers[i](x)
            x = self.conv_activation(x)
            x = self.pool(x)
            x = self.dropout_conv(x)
        x = x.view(x.size(0), -1)  # Flatten the features.
        x = self.dense_activation(self.fc1(x))
        x = self.dropout_fc(x)
        x = self.fc2(x)  # Raw logits (F.cross_entropy applies softmax internally)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("val_loss", loss, prog_bar=True,on_epoch=True)
        self.log("val_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("test_loss", loss,on_epoch=True,on_step=False)
        self.log("test_acc", acc, on_step=False, on_epoch=True, )
        return loss

    def configure_optimizers(self):
        # Choose the optimizer based on the hyperparameter.
        optimizer_type = self.hparams.optimizer_type.lower()
        weight_decay = getattr(self.hparams, "weight_decay", 1e-4)
        if optimizer_type == "adam":
            optimizer = Adam(self.parameters(), lr=self.hparams.learning_rate,weight_decay = weight_decay)
            
        elif optimizer_type == "nadam":
            optimizer = torch.optim.NAdam(self.parameters(), lr=self.hparams.learning_rate,weight_decay = weight_decay)
             
        else:
            raise ValueError(f"Unknown optimizer type: {optimizer_type}")
        return optimizer

# -------------------------------
# Data Splitting with Stratified Sampling
# -------------------------------
def get_train_val_split(dataset, test_size=0.2, random_state=42):
    """
    Given an ImageFolder dataset, splits the indices into training (80%) and
    validation (20%) sets using stratified sampling.
    """
    targets = dataset.targets
    indices = list(range(len(dataset)))
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    train_idx, val_idx = next(sss.split(indices, targets))
    return train_idx, val_idx

# -------------------------------
# Data Transforms (with or without augmentation)
# -------------------------------
def get_transforms(use_data_aug):
    if use_data_aug:
        transform_train = transforms.Compose([
            transforms.Resize((128,128)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(20),
            transforms.ToTensor(),
        ])
    else:
        transform_train = transforms.Compose([
            transforms.Resize((128,128)),
            transforms.ToTensor(),
        ])
    # For test/validation, use minimal transforms.
    transform_test = transforms.Compose([
        transforms.Resize((128,128)),
        transforms.ToTensor(),
    ])
    return transform_train, transform_test

# -------------------------------
# Main Training Function (sweep-compatible)
# -------------------------------
def run_training():
    wandb.init()
    config = wandb.config

    # Process filter organization and kernel size.
    kernel_size = config.get("kernel_size", 3)
    filt_org = config.get("filter_organization", "constant")
    if filt_org == "constant":
        conv_configs = [(config.constant_filter, kernel_size)] * 5
    elif filt_org == "doubling":
        base = config.base_filter
        conv_configs = [(base * (2 ** i), kernel_size) for i in range(5)]
    elif filt_org == "halving":
        base = config.base_filter
        conv_configs = [(base // (2 ** i), kernel_size) for i in range(5)]
    else:
        raise ValueError("Unknown filter organization!")
    
    # Data augmentation setup.
    use_data_aug = config.get("data_augmentation", True)
    transform_train, transform_test = get_transforms(use_data_aug)

    # Load the full training dataset from train_data_path.
    train_dataset_full = ImageFolder(root=config.train_data_path, transform=transform_train)
    # Split the full training dataset into training (80%) and validation (20%).
    train_idx, val_idx = get_train_val_split(train_dataset_full, test_size=0.2, random_state=42)
    train_dataset = Subset(train_dataset_full, train_idx)
    val_dataset = Subset(train_dataset_full, val_idx)

    # Load the test dataset from test_data_path.
    test_dataset = ImageFolder(root=config.test_data_path, transform=transform_test)

    # Create data loaders.
    train_loader = DataLoader(train_dataset, batch_size=int(config.batch_size), shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=int(config.batch_size), shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=int(config.batch_size), shuffle=False, num_workers=4)

    # Create the model.
    model = iNaturalistCNN(
        num_classes=10,
        learning_rate=config.learning_rate,
        input_size=128,
        conv_configs=conv_configs,
        dense_neurons=int(config.dense_neurons),
        dropout_rate=config.dropout_rate,
        conv_activation=config.conv_activation,
        dense_activation=config.dense_activation,
        use_batch_norm=config.batch_norm,
        optimizer_type=config.optimizer  # Use the optimizer hyperparameter here.
    )
    trainer.fit(model, train_loader, val_loader)

    # Set up the Wandb logger.
    wandb_logger = WandbLogger(project=config.project_name)
        
    # Define early stopping
    early_stop_callback = EarlyStopping(
        monitor="val_loss",       # metric to watch
        patience=3,               # number of epochs with no improvement after which training will be stopped
        mode="min",               # because we want to minimize val_loss
        verbose=True
    )
    # Define the PyTorch Lightning trainer.
    trainer = pl.Trainer(
        max_epochs=int(config.max_epochs),
        accelerator="auto",
        devices=1,
        logger=wandb_logger,
    )
    
    # Train the model using training and validation data.
    trainer.fit(model, train_loader, val_loader)
    
    # Test the final model on the test dataset.
    trainer.test(model, dataloaders=test_loader)
    wandb.finish()

In [1]:
# -------------------------------
# Define Sweep Configuration as a Dictionary (no external YAML)
# -------------------------------
sweep_config = {
    "program": "Train.py",  # Ensure this matches your script filename.
    "method": "bayes",      # Options: "grid", "random", "bayes"
    "metric": {
        "name": "val_loss",
        "goal": "minimize"
    },
    "parameters": {
        "filter_organization": {"values": ["constant", "doubling", "halving"]},
        "constant_filter": {"values": [32, 64]},
        "base_filter": {"values": [64,128,]},
        "kernel_size": {"values": [3, 5, 7]},
        "conv_activation": {"values": ["relu", "gelu", "silu", "mish"]},
        "dense_activation": {"values": ["relu", "gelu", "silu", "mish"]},
        "data_augmentation": {"values": [True, False]},
        "batch_norm": {"values": [True, False]},
        "dropout_rate": {"values": [0,0.25]},
        "learning_rate": {"values": [0.0001, 0.001]},
        "dense_neurons": {"values": [256,512]},
        "batch_size": {"values": [32,64]},
        "optimizer": {"values": ["adam"]},
        "max_epochs": {"value": 20 },
        "weight_decay": {"values": [0.0, 1e-4,1e-3]},
        "train_data_path": {"value": "/home/user/kartikey_phd/DA6401/nature_12K/inaturalist_12K/train"},  # Update with actual path.
        "test_data_path": {"value": "/home/user/kartikey_phd/DA6401/nature_12K/inaturalist_12K/val"},    # Update with actual path.
        "project_name": {"value": "iNaturalist_Sweep_final_2"}
    }
}

# -------------------------------
# Run the Sweep and Launch the Agent 
# -------------------------------
if __name__ == "__main__":
    # Initialize the sweep programmatically.
    sweep_id = wandb.sweep(sweep_config, project=sweep_config["parameters"]["project_name"]["value"])
    # Launch the wandb agent to run the training function over multiple trials.
    wandb.agent(sweep_id, function=run_training)


NameError: name 'wandb' is not defined

In [28]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import os
import random
import torch.nn.functional as F
import wandb  # Added for W&B integration

# Initialize W&B run
wandb.init(
    project="inaturalist-predictions",
    config={
        "dataset": "iNaturalist_12K_val",
        "model": "iNaturalistCNN",
        "image_size": (128, 128),
        "num_classes": 10,
        "checkpoint_path": "/home/user/Documents/iNaturalist_Sweep_final_2/2gjdpq1g/checkpoints/epoch=19-step=2500.ckpt"
    }
)

# Define test transforms used during training
transform_test = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Load test dataset (for prediction and class names)
test_data_path = "/home/user/kartikey_phd/DA6401/nature_12K/inaturalist_12K/val"  # replace with your actual path
test_dataset = ImageFolder(root=test_data_path, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# For visualization (raw PIL images)
test_dataset_display = ImageFolder(root=test_data_path, transform=None)

# Verify that both datasets have the same order
assert len(test_dataset) == len(test_dataset_display), "Dataset lengths do not match!"
assert test_dataset.classes == test_dataset_display.classes, "Class names do not match!"
assert len(test_dataset.classes) == 10, "Expected 10 classes in the dataset!"

# Class names (from folder names)
class_names = test_dataset.classes
print("Class names:", class_names)
wandb.config.update({"classes": class_names})  # Log class names

# Load the best model
# from your_model_file import iNaturalistCNN  # Uncomment and update if needed
best_model_path = "/home/user/Documents/iNaturalist_Sweep_final_2/2gjdpq1g/checkpoints/epoch=19-step=2500.ckpt"
best_model = iNaturalistCNN.load_from_checkpoint(best_model_path)
best_model.eval()

# Save model checkpoint as a W&B artifact
model_artifact = wandb.Artifact("best_model_checkpoint", type="model")
model_artifact.add_file(best_model_path)
wandb.log_artifact(model_artifact)

# Store all predictions temporarily
all_correct_samples = {i: [] for i in range(len(class_names))}
all_incorrect_samples = {i: [] for i in range(len(class_names))}

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)

# Create a table for predictions
prediction_table = wandb.Table(columns=["Image", "True Label", "Predicted Label", "Confidence", "Correct"])

# Collect all predictions
with torch.no_grad():
    for idx, ((x_tensor, true_label), (img_pil, true_label_display)) in enumerate(zip(test_loader, test_dataset_display)):
        assert true_label.item() == true_label_display, f"True label mismatch at index {idx}!"
        x_tensor = x_tensor.to(device)
        logits = best_model(x_tensor)
        pred_label = torch.argmax(logits, dim=1).item()
        probs = F.softmax(logits, dim=1)
        confidence = probs[0, pred_label].item() * 100

        print(f"Index {idx}: True={class_names[true_label.item()]}, Pred={class_names[pred_label]}, Confidence={confidence:.2f}%")

        # Log to W&B table
        is_correct = pred_label == true_label.item()
        img_wandb = wandb.Image(img_pil, caption=f"True: {class_names[true_label.item()]}, Pred: {class_names[pred_label]}")
        prediction_table.add_data(
            img_wandb,
            class_names[true_label.item()],
            class_names[pred_label],
            f"{confidence:.2f}%",
            is_correct
        )

        sample = (img_pil, true_label.item(), pred_label, confidence)
        if is_correct:
            all_correct_samples[true_label.item()].append(sample)
        else:
            all_incorrect_samples[true_label.item()].append(sample)

# Log prediction table
wandb.log({"prediction_samples": prediction_table})

# Randomly select up to 3 correct and 3 incorrect samples per class
correct_samples = {i: [] for i in range(len(class_names))}
incorrect_samples = {i: [] for i in range(len(class_names))}

random.seed(42)
for class_idx in range(len(class_names)):
    if len(all_correct_samples[class_idx]) > 0:
        correct_samples[class_idx] = random.sample(
            all_correct_samples[class_idx],
            min(3, len(all_correct_samples[class_idx]))
        )
    if len(all_incorrect_samples[class_idx]) > 0:
        incorrect_samples[class_idx] = random.sample(
            all_incorrect_samples[class_idx],
            min(3, len(all_incorrect_samples[class_idx]))
        )

# Create the 10x3 grid
fig, axes = plt.subplots(nrows=10, ncols=3, figsize=(15, 30), facecolor='#f0f0f5')
plt.subplots_adjust(hspace=0.4, wspace=0.3, left=0.15)

for class_idx in range(10):
    all_samples = correct_samples[class_idx] + incorrect_samples[class_idx]
    random.shuffle(all_samples)
    samples = all_samples[:3]

    for i in range(3):
        ax = axes[class_idx, i]
        ax.axis("off")
        if i >= len(samples):
            ax.set_title("N/A", fontsize=10, color='gray')
            continue
        img_pil, true_label, pred_label, confidence = samples[i]

        ax.imshow(img_pil)
        ax.set_facecolor('#ffffff')
        is_correct = (pred_label == true_label)
        border_color = '#00cc00' if is_correct else '#ff3333'
        rect = patches.Rectangle((0, 0), img_pil.width-1, img_pil.height-1, linewidth=6,
                                 edgecolor=border_color, facecolor='none', alpha=0.8)
        ax.add_patch(rect)
        title = f"True: {class_names[true_label]}\nPred: {class_names[pred_label]}\nConf: {confidence:.1f}%"
        ax.set_title(title, fontsize=9, color='#333333', pad=10, bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

    fig.text(0.05, 0.5 * (19 - class_idx) / 20 + 0.025, class_names[class_idx],
             fontsize=12, fontweight='bold', color='#333333', va='center', rotation=90)

plt.suptitle("Model Predictions on iNaturalist Test Set\nGreen = Correct, Red = Incorrect", fontsize=18, fontweight='bold', y=0.99, color='#333333')
fig.text(0.5, 0.01, "Legend: Each row represents a class. Images show true labels, predicted labels, and confidence scores.\nGreen borders indicate correct predictions; red borders indicate incorrect ones.",
         fontsize=10, ha='center', color='#555555', bbox=dict(facecolor='white', alpha=0.8, edgecolor='gray'))

# Save and log the figure to W&B
plt.tight_layout(rect=[0.15, 0.03, 1, 0.98])
wandb.log({"prediction_grid": wandb.Image(fig)})
plt.close(fig)  # Close to free memory

# Finish W&B run
wandb.finish()

Class names: ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Index 0: True=Amphibia, Pred=Reptilia, Confidence=24.37%
Index 1: True=Amphibia, Pred=Mollusca, Confidence=22.80%
Index 2: True=Amphibia, Pred=Amphibia, Confidence=32.38%
Index 3: True=Amphibia, Pred=Fungi, Confidence=29.34%
Index 4: True=Amphibia, Pred=Plantae, Confidence=28.68%
Index 5: True=Amphibia, Pred=Fungi, Confidence=24.42%
Index 6: True=Amphibia, Pred=Arachnida, Confidence=35.17%
Index 7: True=Amphibia, Pred=Insecta, Confidence=40.30%
Index 8: True=Amphibia, Pred=Mollusca, Confidence=22.40%
Index 9: True=Amphibia, Pred=Animalia, Confidence=33.31%
Index 10: True=Amphibia, Pred=Reptilia, Confidence=35.15%
Index 11: True=Amphibia, Pred=Animalia, Confidence=32.96%
Index 12: True=Amphibia, Pred=Reptilia, Confidence=27.20%
Index 13: True=Amphibia, Pred=Arachnida, Confidence=24.17%
Index 14: True=Amphibia, Pred=Amphibia, Confidence=36.16%
Index 15: True=Amphi